# Convexity of monomial expressions

In this document, we study the convexity of monomial expressions like $x,y \mapsto x^a.y^b$
on the positive orthant ($x,y>0$).
The goal is to find conditions (e.g. on coefficients $a,b$) under which such expression is convex.

PH, December 2020

In [4]:
import sympy
from sympy import symbols, Function, Matrix, simplify, invert

In [7]:
a,b,c = symbols('a b c')
x,y,z = symbols('x y z', positive=True)

In [11]:
def Hessian(f, *args):
    """returns Hessian of expression `f`, with variables specified by `*args`
    
    Example:
    >>> Hessian(x**3 * y**2, x, y)
    Matrix([[6*x*y**2, 6*x**2*y], [6*x**2*y, 2*x**3]])
    """
    H = [[f.diff(xi).diff(xj) for xi in args] for xj in args]
    H = Matrix(H)
    H = simplify(H)
    return H

## 2D separable expression, linear in $x$

This is section deals with a more general case than monomial expression.
We consider a 2D separable expression which is linear in $x$:

$$f x,y \mapsto x.\rho(y)$$

i.e. this corresponds to the case $a=1$ in the next section,
but the dependency in $y$ is any function $\rho$, rather than the power law $y\mapsto y^b$.

The Hessian of $f$ is 

$$H(x,y) = 
\begin{bmatrix}
    0        &  \rho'(y) \\
    \rho'(y) & x\rho''(y)
\end{bmatrix}
$$

The conditions for $H$ to be definite semi-positive (for all $x,y>0$) are:
* non-negative diagonal terms, i.e. $\rho''>0$, that is $\rho$ should be convex
* non-negative determinant

The 2nd condition is:

$$0 - \rho'^2 \geq 0$$

which becomes $\rho' = 0$.

This means that only **constant** $rho$ functions are allowed for the $x.\rho(y)$ to be convex.

Translated in the context of storage loss models, this means that among PWL-in-$P$ models,
only *constant efficiency charge/discharge coefficients* yield convex expressions.
In particular, energy dependent efficiency coefficients ($\eta(E)$), even if convex in $E$,
do not yield convex loss models.

In [13]:
ρ = Function('ρ')
Hessian(ρ(y)*x, x,y)

Matrix([
[                  0,        Derivative(ρ(y), y)],
[Derivative(ρ(y), y), x*Derivative(ρ(y), (y, 2))]])

In [14]:
Hessian(ρ(y)*x, x,y).det()

-Derivative(ρ(y), y)**2

## 2D monomial 

E.g useful for loss model dependent in power ($x$) and energy ($y$) or any other 2nd variable (temperature, aging).
In this context, coefficient $a$ can be supposed positive (losses increase with $x$).

$f_2: x,y \mapsto x^a.y^b$

In [15]:
f2 = x**a * y**b
f2

x**a*y**b

In [16]:
H2 = Hessian(f2, x, y)
H2

Matrix([
[a*x**(a - 2)*y**b*(a - 1), a*b*x**(a - 1)*y**(b - 1)],
[a*b*x**(a - 1)*y**(b - 1), b*x**a*y**(b - 2)*(b - 1)]])

The Hessian has a simpler expression if divided by $f_2$:

In [17]:
simplify(H2/f2)

Matrix([
[a*(a - 1)/x**2,      a*b/(x*y)],
[     a*b/(x*y), b*(b - 1)/y**2]])

Perhaps Hessian is even simpler if multiplied by $\frac{xy}{f}$

In [18]:
simplify(H2*x*y/f2)

Matrix([
[a*y*(a - 1)/x,           a*b],
[          a*b, b*x*(b - 1)/y]])

Determinant:

$$\det  \frac{xy}{f} H_2 = ab(1-a-b)$$

so we have

$$\det  H_2 = \frac{f_2^2}{x^2y^2}[ab(1-a-b)]$$

(reminder: $\det kA = k^n \det A$, where $n$ is the dimention of matrix $A$)

In [36]:
d2 = simplify((H2*x*y/f2).det())
d2

a*b*(-a - b + 1)

**TODO**: 

- Write down convexity proof here
- Plot 2D heatmap of the determinant, to show where it is positive
- Insert png version of the illustration:

![Convexity of f2 illustrated in ab plane](ab_convexity.svg)

### Inverse of Hessian

Inverse of $H_2$ (needed for next section):

again the same trick of multiplying by $f^{-1}$ or $\frac{xy}{f}$ makes the expression simpler:

In [21]:
simplify((H2/f2).inverse())

Matrix([
[x**2*(1 - b)/(a*(a + b - 1)),              x*y/(a + b - 1)],
[             x*y/(a + b - 1), y**2*(1 - a)/(b*(a + b - 1))]])

multiply by $ab(1-a-b)$ (det of $H_2 x y/f_2$):

In [38]:
simplify((H2/f2).inverse()*d2)

Matrix([
[b*x**2*(b - 1),       -a*b*x*y],
[      -a*b*x*y, a*y**2*(a - 1)]])

In [22]:
simplify((H2*x*y/f2).inverse())

Matrix([
[x*(1 - b)/(a*y*(a + b - 1)),               1/(a + b - 1)],
[              1/(a + b - 1), y*(1 - a)/(b*x*(a + b - 1))]])

## 3D monomial

$f_3: x,y,z \mapsto x^a.y^b.z^c$

For convexity analysis, we will build on the convexity of the 2D case, by using [Conditions for semi-definiteness using Schur complement](https://en.wikipedia.org/wiki/Schur_complement#Conditions_for_positive_definiteness_and_semi-definiteness).

In [24]:
f3 = x**a * y**b * z**c
f3

x**a*y**b*z**c

In [25]:
H3 = Hessian(f3, x, y, z)
H3

Matrix([
[a*x**(a - 2)*y**b*z**c*(a - 1), a*b*x**(a - 1)*y**(b - 1)*z**c, a*c*x**(a - 1)*y**b*z**(c - 1)],
[a*b*x**(a - 1)*y**(b - 1)*z**c, b*x**a*y**(b - 2)*z**c*(b - 1), b*c*x**a*y**(b - 1)*z**(c - 1)],
[a*c*x**(a - 1)*y**b*z**(c - 1), b*c*x**a*y**(b - 1)*z**(c - 1), c*x**a*y**b*z**(c - 2)*(c - 1)]])

Again same trick is to consider $H/f$ to get simpler expressions:

In [28]:
J3 = simplify(H3/f3)
J3

Matrix([
[a*(a - 1)/x**2,      a*b/(x*y),      a*c/(x*z)],
[     a*b/(x*y), b*(b - 1)/y**2,      b*c/(y*z)],
[     a*c/(x*z),      b*c/(y*z), c*(c - 1)/z**2]])

We split the matrix into blocks to apply the Schur complement based method

$$J = H_3/f = \begin{bmatrix}
    A   &B\\
    B^T &d
\end{bmatrix}
$$

with:
* $A = H_2/f_2$, studied above
* $B^T = [ab/xy, ac/xz]$
* $d = c(c-1)/z^2$

We use the following result: 

If $A\succ 0$ (equiv. to $\succeq 0$ and invertible?),
then:

> $J \succeq 0$ is equivalent to $J/A\succeq 0$

where $$J/A = d - B^T A^{-1} B$$ is the _Schur complement_ of block $A$ of matrix $J$.
Because of the way we split $J$, it is a _scalar_ ($1\times1$).

So we have to compute this Schur complement.

First, extract the three blocks of $J$.

In [29]:
A = J3[0:2,0:2] 
A #  == H2/f2

Matrix([
[a*(a - 1)/x**2,      a*b/(x*y)],
[     a*b/(x*y), b*(b - 1)/y**2]])

In [30]:
simplify(H2/f2)

Matrix([
[a*(a - 1)/x**2,      a*b/(x*y)],
[     a*b/(x*y), b*(b - 1)/y**2]])

In [31]:
d = J3[2,2]
d

c*(c - 1)/z**2

In [33]:
B = J3[0:2,2]
B

Matrix([
[a*c/(x*z)],
[b*c/(y*z)]])

### Computation of Schur complement $J/A$

step by step:
1. Invert $A$
2. Multiply with $B^T$ and $B$

In [34]:
Ainv = simplify(A.inv())
Ainv

Matrix([
[x**2*(1 - b)/(a*(a + b - 1)),              x*y/(a + b - 1)],
[             x*y/(a + b - 1), y**2*(1 - a)/(b*(a + b - 1))]])

In [35]:
simplify(Ainv*B)

Matrix([
[c*x/(z*(a + b - 1))],
[c*y/(z*(a + b - 1))]])

Clarify the expression:

$A^{-1}.B = -\frac{c}{z(1-a-b)} (x,y)^T$

In [42]:
simplify(Ainv*B /(-c/z/(1-a-b))) 

Matrix([
[x],
[y]])

$B^TA^{-1}.B$ is quite simple:

In [64]:
BtAiB = simplify((B.T*Ainv*B)[0,0])
BtAiB

c**2*(a + b)/(z**2*(a + b - 1))

Finally, the Schur complement: (which need to be positive for convexity)

In [68]:
S = simplify(d - BtAiB)
S 

-c*(a + b + c - 1)/(z**2*(a + b - 1))

Remove positive factors $z^2$ and $(a+b-1)$: (2nd one TO BE CHECKED)

In [71]:
simplify(S * z**2 * (a+b-1))

c*(-a - b - c + 1)

Conclusion: (TO BE CHECKED and WRITTEN MORE CLEARLY)

for $c<0$:

$$a+b+c \geq 1$$